## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-14-13-11-57 +0000,nypost,Drone Ranger: NYPD primed to start defending a...,https://nypost.com/2026/02/14/us-news/nypd-pri...
1,2026-02-14-13-09-51 +0000,nypost,American wounded in Benghazi — and still carry...,https://nypost.com/2026/02/14/us-news/american...
2,2026-02-14-13-08-02 +0000,nypost,Ousted Columbia student who declared ‘Zionists...,https://nypost.com/2026/02/14/us-news/suspende...
3,2026-02-14-13-07-23 +0000,nypost,Glowing Greenland: Danes who cleaned up a US n...,https://nypost.com/2026/02/14/us-news/cancer-s...
4,2026-02-14-13-05-06 +0000,nypost,Doguefight: Vogue sues tiny dog mag in David a...,https://nypost.com/2026/02/14/us-news/vogue-su...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2493/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,39
67,ice,19
127,epstein,18
81,new,17
156,shooting,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...,74
260,2026-02-13-18-33-12 +0000,nypost,AOC downplays 2028 talk in Munich — babbles wh...,https://nypost.com/2026/02/13/us-news/aoc-does...,71
190,2026-02-13-21-44-46 +0000,nypost,Ted Lieu makes wild claim about Trump and Epst...,https://nypost.com/2026/02/13/us-news/lieu-mak...,71
165,2026-02-13-22-30-21 +0000,nyt,"At Munich Summit, European Leaders Agree With ...",https://www.nytimes.com/2026/02/13/world/europ...,68
33,2026-02-14-11-00-57 +0000,nyt,Democrats in Illinois Senate Primary Debate Ho...,https://www.nytimes.com/2026/02/14/us/politics...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,74,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...
73,62,2026-02-14-06-47-49 +0000,nypost,White House unveils cheeky Valentine’s Day car...,https://nypost.com/2026/02/14/us-news/white-ho...
274,57,2026-02-13-17-39-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
177,53,2026-02-13-22-13-46 +0000,nypost,Here’s the crude email exchange with Jeffrey E...,https://nypost.com/2026/02/13/business/heres-t...
158,50,2026-02-13-22-41-16 +0000,startribune,DHS says ICE agents suspended for ‘untruthful’...,https://www.startribune.com/dhs-says-ice-agent...
209,45,2026-02-13-20-53-59 +0000,bbc,Ex-CNN host Don Lemon pleads not guilty in Min...,https://www.bbc.com/news/articles/cge89r1e158o...
153,38,2026-02-13-22-59-00 +0000,cbc,Labelling Palestine Action a terrorist group w...,https://www.cbc.ca/radio/asithappens/palestine...
27,34,2026-02-14-12-00-00 +0000,latimes,Lopez: In 50-year fight to protect California'...,https://www.latimes.com/california/story/2026-...
279,33,2026-02-13-17-11-14 +0000,latimes,San Francisco teachers' strike ends as union a...,https://www.latimes.com/california/story/2026-...
260,32,2026-02-13-18-33-12 +0000,nypost,AOC downplays 2028 talk in Munich — babbles wh...,https://nypost.com/2026/02/13/us-news/aoc-does...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
